In [ ]:
import json

# Load the list from the JSON file
with open('speech_total.json', 'r') as file:
    speech_total = json.load(file)

len(speech_total)

In [ ]:
# Setting up Groq API
import os
from dotenv import load_dotenv
from groq import Groq

load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API"))

def get_groq_response(query):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        model="mixtral-8x7b-32768",
        temperature=0.5,
        max_tokens=1024,
    )
    return chat_completion.choices[0].message.content

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import faiss
import pickle
import torch

# Step 1: Load the embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embedding
embeddings = embed_model.encode(speech_total, show_progress_bar=True)
print(f"Embeddings shape: {embeddings.shape}")

# Step 2: Create and save FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype('float32'))

# Save the index and texts
faiss.write_index(index, "faiss_index.bin")
with open('original_texts.pkl', 'wb') as f:
    pickle.dump(speech_total, f)

# Step 3: Load the language model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set the pad token to be the end of sequence token
tokenizer.pad_token = tokenizer.eos_token

# Check if GPU is available, and if so set up device for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Create a text generation pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# Create function to retrieve similar context from FAISS index based on query given
def retrieve_context(query, k=3):
    query_embedding = embed_model.encode([query])[0]
    D, I = index.search(np.array([query_embedding]).astype('float32'), k)
    retrieved_texts = [speech_total[i] for i in I[0]]
    return " ".join(retrieved_texts)

# Create function to generate a response, by combining retrieved context and response from Groq, then making it sound like Elon
def generate_response(query, max_new_tokens=200):
    # Get factual response from Groq
    groq_response = get_groq_response(query)
    
    # Retrieve context from your existing data
    context = retrieve_context(query)
    
    # Combine Groq response with retrieved context
    combined_input = f"Context: {context}\n\nGroq Response: {groq_response}\n\nQuery: {query}\n\nResponse:"
    
    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors="pt", padding=True, truncation=True, max_length=512)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    
    # Generate Elon-style response
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_text = response.split("Response:")[-1].strip()
    
    # Call Elonify (function below) to make response sound like Elon
    return elonify(generated_text)

# Function to make response sound like Elon
def elonify(content):
    # Ask our trained language model to rewrite following answer into Elon sounding answer
    prompt = f"""
    Rewrite the following content in Elon Musk's speaking style, based on the training data:

    {content}

    Elon Musk's response:
    """
    
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=1000) # If desire longer input, change max_length
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=1000, # Token size for answer
        num_return_sequences=1, # Number of responses
        no_repeat_ngram_size=2, # Max 2 consecutive words
        temperature=0.7, # Randomness
        pad_token_id=tokenizer.eos_token_id # Set padding token to end of sequence token
    )
    
    # Convert model output back into text, takes response
    return tokenizer.decode(output[0], skip_special_tokens=True).split("Elon Musk's response:")[-1].strip()



In [ ]:
# Testing and Improving RAG System

import random # Used for sampling
from tqdm import tqdm # Provides progress bars

def evaluate_rag_system(num_samples=100):
    # Sample queries or create a test set
    test_queries = [
        "What does Elon Musk think about renewable energy?",
        "How does Elon Musk view the future of space exploration?",
        # Add more diverse queries here
    ]

    results = []
    for query in tqdm(random.sample(test_queries, min(num_samples, len(test_queries)))):
        response = generate_response(query)
        results.append({
            "query": query,
            "response": response,
            # You might add more metrics here, like response time, etc.
        })

    # Here you would typically add code to calculate metrics
    # such as relevance, coherence, factual accuracy, etc.
    # This often requires human evaluation or comparison against known ground truths

    return results

# Run evaluation
eval_results = evaluate_rag_system()
print(f"Evaluated {len(eval_results)} queries")
# Add code here to analyze and display results

In [ ]:
from IPython.display import clear_output

def interactive_qa():
    while True:
        query = input("Enter your question (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            print("Thank you for using ElonAI, goodbye!")
            break
        
        if query.strip():
            print(f"\nQ: {query}")
            try:
                response = generate_response(query)
                print(f"\nA: {response}\n")
            except Exception as e:
                print(f"An error occurred: {str(e)}\n")
        else:
            print("Please enter a question.\n")

print("Type your question and press Enter to speak with.")
interactive_qa()